# Import libraries

In [2]:
import os
from flask import Flask, render_template, request, g, redirect, url_for,flash
from werkzeug import secure_filename
from pathlib import Path
import sqlite3
from pybtex.database import parse_file

1. os -- file pathing, saving files
2. flask -- we're making a web application
3. werkzeug -- examples suggested that this was more secure for the user
4. pathlib -- we'll use this to check for the database file
5. sqlite3 -- making a database, using SQL queries
6. pybtex -- parsing the .bib file

# Declarations

In [3]:
app = Flask(__name__)

app.config['UPLOAD_FOLDER'] = 'uploads'
db_path = os.path.join(app.root_path, 'uploads/biblio.db')
ALLOWED_EXTENSIONS = set(['bib'])

db_path is the path to the database, stored in the 'uploads' directory;  
ALLOWED_EXTENSIONS will limit the user to the uploading of .bib files only

# Function definitions

## Database functions

In [4]:
def get_db():
    connection = sqlite3.connect(db_path)
    cursor = connection.cursor()
    return connection, cursor  #connection and cursor pointing to the database

def create_db():
    connection = sqlite3.connect(db_path)
    cursor = connection.cursor()
    
    #create the table
    cursor.execute('''CREATE TABLE bib
             (tag text, authors text, journal text, volume int, pages text, year int, title text, collection text)''')
    #and save it
    connection.commit()
    return connection, cursor


def query_db(query_string):
    connection, cursor = get_db()
    cursor.execute(f'SELECT * FROM bib WHERE '+ query_string)
    all_rows = cursor.fetchall()
    return all_rows #a list of database entries matching the query criterion

create_db() is called when the user first enters the 'index' URL; a database table with 8 columns is created  
get_db() can be called to get the connection/cursor of the database in order to manipulate the table  
query_db() is used to query the database using the (partial) SQL string provided by the user

In [5]:
def fill(bib_data, collection):
    connection, cursor = get_db()
    for entry in bib_data.entries: 
        #get the authors
        authors = get_db_authors(bib_data, entry)
        #get everything else
        tag,journal,volume,pages,year,title= entry, get_db_string('journal', bib_data, entry),\
        get_db_int('volume', bib_data, entry), get_db_string('pages', bib_data, entry), \
        get_db_int('year', bib_data, entry), get_db_string('title', bib_data, entry)
        
        #we'll clean up the title/journal strings a bit
        title = title.replace('{', '')
        title = title.replace('}', '')
        journal = journal.replace("\\", '')
        
        #populate the database
        exe_string = f"INSERT INTO bib VALUES ('{tag}', '{authors}', '{journal}', '{volume}', '{pages}', '{year}', '{title}', '{collection}')"
        cursor.execute(exe_string)
    #and save the changes
    connection.commit()

def get_db_string(key, bib_data, entry):
    try:
        return bib_data.entries[entry].fields[key]
    except KeyError:
        return ''  #when there's no journal, title, etc. for this entry
    
def get_db_int(key, bib_data, entry):
    try:
        return int(bib_data.entries[entry].fields[key])
    except KeyError:
        return 0   #when there's no year or volume for this entry
    
def get_db_authors(bib_data, entry):
    authors = ''
    try: 
        for author in bib_data.entries[entry].persons['author']:
            new_author = str(author)
            
            #we'll clean up the author names a bit
            characters = ['{', '}', "'",'"', '\\', '//']
            for c in characters:
                new_author = new_author.replace(c, '')
            authors += new_author + ' and '
        authors = authors[:-5]  #remove final 'and'
        
    except KeyError:
        authors = 'no authors'   #when there's no author listed for this entry
        
    return authors  #a string with all of the authors listed

fill() is called to populate the database table with entries from the bibliography.

In order to get the data for the table, the parsed .bib file must be correctly read.  To do this, three get_db...() functions are created.  They correctly handle string (e.g. journal) and integer (e.g. year) values from the bibliography, along with creating an author list string.  When an entry is missing a key, the appropriate 'empty' value (empty string, 0, or 'no authors') is returned to be placed in the table.

## Misc. Functions

In [6]:
def allowed_file(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS
    #truth value of appropriate file type
    
def get_collections(all_rows):
    output = []
    for row in all_rows:
        if row[7] not in output:
            output.append(row[7])
    return output  #a list of unique collection names

allowed_file() checks that the file requested to be uploaded is .bib  
get_collections() finds unique collection names in the database

# URL routes and their functions

In [7]:
@app.route('/')
@app.route('/index')
def index():
    if Path(db_path).is_file():  #if there's a database,
        db = 1
        connection, cursor = get_db()
        cursor.execute(f'SELECT * FROM bib')  #look at all entries
        all_rows = cursor.fetchall()
        collections = get_collections(all_rows)  #and find unique collection names
        
    #if there's not a database
    else:
        db = 0
        connection, cursor = create_db()  #create one
        all_rows = cursor.fetchall()
        collections = get_collections(all_rows)  #and get an empty list of collections
    return render_template('index.html', db = db, collections = collections)

This is the 'home' page function of our web application.  If the database exists, it displays the unique collection names (if any).  If not, a database is created.

In [ ]:
@app.route('/upload')
def upload():
    message = request.args.get('message')  #a message about recent upload attempts will be displayed to the user
    if message == None:
        message = ''
    return render_template('upload.html', message = message)

@app.route('/uploader', methods = ['GET', 'POST'])
def upload_file():
    if request.method == 'POST':
        #Check for file selection and appropriate file type
        if 'file' not in request.files:
            return redirect(url_for('upload', message = 'Please select a file.'))
        file = request.files['file']
        if file.filename == '':
            return redirect(url_for('upload', message = 'Please select a file with a name.'))
        if not allowed_file(file.filename):
            return redirect(url_for('upload', message = 'Please select a .bib file.'))
        
        #if everything looks good, upload the selected file
        else:
            file.save(os.path.join(app.config['UPLOAD_FOLDER'], secure_filename(file.filename)))
            collection = request.form['collection']
            bib_data = parse_file(os.path.join(app.config['UPLOAD_FOLDER'], secure_filename(file.filename)))  #parse the .bib file
            fill(bib_data, collection)   #populate the database
            return redirect(url_for('upload', message = 'Upload successful!'))

These are the 'upload' page functions of our web application.  upload() renders the upload.html file, with a message depending on recent upload attempts.  The upload_file() function is called when an upload attempt is made.  After checking for the appropriate file, the file is uploaded, parsed by pybtex, and used to fill the database.

In [ ]:
@app.route('/query')
def query():
    query_string = request.args.get('query_string')
    if query_string == None:
        results = []
    else:
        try:
            results = query_db(query_string)   #a good query string will return a list of matching database entries
        except sqlite3.OperationalError:  #the user may enter a non-sensical query
            results = ['error']
    return render_template('query.html', results = results)

@app.route('/query-er', methods = ['GET', 'POST'])  #following the upload/uploader naming convention
def upload_query():
    if request.method == 'POST':
        query_string = request.form['query_string']
        return redirect(url_for('query', query_string = query_string))

These are the 'query' page functions of our web application. query() renders the query.html file, while upload_query() is called when the user submits a (partial) SQL query string.

# Run the app

In [ ]:
if __name__ == '__main__':
    app.run()

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [17/Apr/2018 05:55:55] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Apr/2018 05:55:58] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Apr/2018 05:56:01] "GET /upload HTTP/1.1" 200 -
127.0.0.1 - - [17/Apr/2018 05:56:06] "POST /uploader HTTP/1.1" 302 -
127.0.0.1 - - [17/Apr/2018 05:56:06] "GET /upload?message=Upload+successful%21 HTTP/1.1" 200 -
127.0.0.1 - - [17/Apr/2018 05:56:09] "GET /index HTTP/1.1" 200 -
127.0.0.1 - - [17/Apr/2018 05:56:11] "GET /upload HTTP/1.1" 200 -
127.0.0.1 - - [17/Apr/2018 05:56:12] "POST /uploader HTTP/1.1" 302 -
127.0.0.1 - - [17/Apr/2018 05:56:12] "GET /upload?message=Please+select+a+file+with+a+name. HTTP/1.1" 200 -
127.0.0.1 - - [17/Apr/2018 05:56:17] "POST /uploader HTTP/1.1" 302 -
127.0.0.1 - - [17/Apr/2018 05:56:17] "GET /upload?message=Please+select+a+file+with+a+name. HTTP/1.1" 200 -
127.0.0.1 - - [17/Apr/2018 05:56:21] "GET /query HTTP/1.1" 200 -
127.0.0.1 - - [17/Apr/2018 05:56:30] "PO